In [13]:

now_utc = datetime.now(timezone.utc)
timestamp_str = now_utc.strftime("%Y-%m-%dT%H-%M-%S.%f")[:-3] + 'Z'

print(timestamp_str + "-" + str(random.randint(0, 999)))


2024-02-14T15-54-04.418Z-916
